In [1]:
%pip install qiskit==1.2.4
%pip install qiskit-aer==0.15.1
%pip install pylatexenc==2.10

   ---------------------------------------- 0.0/4.6 MB ? eta -:--:--
    --------------------------------------- 0.1/4.6 MB 5.5 MB/s eta 0:00:01
   --- ------------------------------------ 0.4/4.6 MB 5.8 MB/s eta 0:00:01
   ------- -------------------------------- 0.8/4.6 MB 6.4 MB/s eta 0:00:01
   ---------- ----------------------------- 1.2/4.6 MB 6.7 MB/s eta 0:00:01
   ------------ --------------------------- 1.5/4.6 MB 6.8 MB/s eta 0:00:01
   --------------- ------------------------ 1.8/4.6 MB 6.5 MB/s eta 0:00:01
   ----------------- ---------------------- 2.0/4.6 MB 6.1 MB/s eta 0:00:01
   -------------------- ------------------- 2.3/4.6 MB 6.1 MB/s eta 0:00:01
   ----------------------- ---------------- 2.7/4.6 MB 6.3 MB/s eta 0:00:01
   ------------------------- -------------- 3.0/4.6 MB 6.3 MB/s eta 0:00:01
   --------------------------- ------------ 3.1/4.6 MB 5.9 MB/s eta 0:00:01
   ----------------------------- ---------- 3.4/4.6 MB 5.8 MB/s eta 0:00:01
   ----------------

In [2]:
from qiskit import QuantumCircuit
from qiskit.converters import circuit_to_gate
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Operator
from qiskit.quantum_info import Statevector
from qiskit import transpile 
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit import ControlledGate
import math 

# The aim of the assignment is to simulate the Ekert91 key distribution protocol.

# This notebook is for a simulation of the protocol with an attacker, to demonstrate that the attacker can be detected.
root2 = math.sqrt(2)

denom1 = math.sqrt(4 + 2*root2)
denom2 = math.sqrt(4 - 2*root2) 

B1_transform_matrix = [ [  1 / denom1 , (1 + root2) / denom1 ],
                        [ -1 / denom2 , (root2 - 1) / denom2 ]]
B0_transform_matrix = [ [ -1 / denom1 , (1 + root2) / denom1 ],
                        [  1 / denom2 , (root2 - 1) / denom2 ]]
def average(c,n):
    backend = BasicSimulator()
    compiled = transpile(c, backend)
    job_sim = backend.run (compiled, shots=n)
    result_sim = job_sim.result() 
    counts = result_sim.get_counts(compiled) 
    print (counts)
    count00 = counts.get('00',0)
    count01 = counts.get('01',0)
    count10 = counts.get('10',0)
    count11 = counts.get('11',0)
    return (count00 - count01 - count10 + count11)/n

def entangledPair():
    q = QuantumCircuit(2,2)
    q.x(1)
    q.h(0)
    q.z(0)
    q.cx(0,1)
    return q

def random():
    q = QuantumCircuit(1) 
    q.h(0) 
    q.measure_all() 
    backend = BasicSimulator()
    compiled = transpile(q, backend)
    job_sim = backend.run(compiled, shots=1)
    result_sim = job_sim.result() 
    counts = result_sim.get_counts(compiled)
    return counts.get("1",0)

N = 100

# x w 
circuit_X_W = entangledPair()
circuit_X_W.h(0)
circuit_X_W.measure(0,0)

#attacker measure 
circuit_X_W.measure(1,1)
circuit_X_W.unitary(B0_transform_matrix,[1])
circuit_X_W.measure(1,1)
avg_XW = average(circuit_X_W,N)
print(avg_XW)

# x v
circuit_X_V = entangledPair() 
circuit_X_V.h(0)
circuit_X_V.measure(0,0)

circuit_X_V.measure(1,1)
circuit_X_V.unitary(B1_transform_matrix,[1])
circuit_X_V.measure(1,1)
avg_XV = average(circuit_X_V,N)
print(avg_XV)

#z w
circuit_Z_W = entangledPair() 
circuit_Z_W.z(0) 
circuit_Z_W.measure(0,0)

circuit_Z_W.measure(1,1)
circuit_Z_W.unitary(B0_transform_matrix,[1])
circuit_Z_W.measure(1,1)
avg_ZW = average(circuit_Z_W,N) 
print(avg_ZW)

#z v
circuit_Z_V = entangledPair()
circuit_Z_V.z(0) 
circuit_Z_V.measure(0,0)

circuit_Z_V.measure(1,1)
circuit_Z_V.unitary(B1_transform_matrix,[1])
circuit_Z_V.measure(1,1)
avg_ZV = average(circuit_Z_V,N)
print(avg_ZV)


# final average 
result = abs(avg_XW - avg_XV + avg_ZW + avg_ZV)
print("s = ",result)

if result < 2:
    print("only classical correlation, attacker present")
elif result < 3.6:
    # between 2 and 3.6
    print("entanglement, no attacker ")

{'10': 25, '01': 23, '11': 21, '00': 31}
0.04
{'10': 29, '01': 23, '00': 23, '11': 25}
-0.04
{'11': 52, '00': 35, '01': 7, '10': 6}
0.74
{'00': 44, '11': 43, '10': 8, '01': 5}
0.74
s =  1.56
only classical correlation, attacker present
